> **This notebook uses a langchain `create_csv_agent` to solve a math problem.**
> - **Thought:** Data is better recognized with a data loader.
> - **Observation:** Results are great, a big improvement from `agent-math.ipynb`

## Environment Setup

In [5]:
import os
from dotenv import load_dotenv

# Set environment variables in the .env file.
load_dotenv()

COMPLETION_MODEL = os.environ["OPENAI_COMPLETION_MODEL"]
COMPLETION_DEPLOYMENT = os.environ["OPENAI_COMPLETION_DEPLOYMENT"]
CHAT_MODEL = os.environ["OPENAI_CHAT_MODEL"]
CHAT_DEPLOYMENT = os.environ["OPENAI_CHAT_DEPLOYMENT"]
OPENAI_API_VERSION = os.environ["OPENAI_API_VERSION"]

In [6]:
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI

llm = AzureOpenAI(
    model_name=COMPLETION_MODEL,
    deployment_name=COMPLETION_DEPLOYMENT,
    #temperature=0.3,
    verbose=True
)

chat = AzureChatOpenAI(
    deployment_name=CHAT_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION,
    #temperature=0.3,
    verbose=True
)

## Code

In [7]:
import csv
with open('data/people.csv', newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)

['Name', 'Gender', 'Age']
['John', 'M', '13']
['Steve', 'M', '25']
['Sarah', 'F', '18']
['Jess', 'F', '45']
['Will', 'M', '50']


In [8]:
from langchain.agents import create_csv_agent
from langchain.agents.agent_types import AgentType

agent = create_csv_agent(
    llm,
    'data/people.csv',
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [19]:
agent.run("""Give me a summary of people by their groups.
Age groups are 0-12 years, 12-18 years, 18-30 years, 30-60 years, and older than 60 years.
Example output:
0-12 years: 0
12-18 years: 2 (John, Jane)
18-30 years: 1 (Bob)
30-60 years: 1 (Mary)
Older than 60 years: 0""")



> Entering new AgentExecutor chain...

Thought: I need to group the people in the dataframe by their age.
Action: python_repl_ast
Action Input: df.groupby('Age')
Observation: <pandas.core.groupby.generic.DataFrameGroupBy object at 0x000001DFABC31050>
Thought: I need to count the number of people in each age group. 
Action: python_repl_ast
Action Input: df.groupby('Age').count()['Name']
Observation: Age
13    1
18    1
25    1
45    1
50    1
Name: Name, dtype: int64
Thought: I now know the final answer.
Final Answer: 
0-12 years: 1 (John)
12-18 years: 1 (Sarah)
18-30 years: 1 (Steve)
30-60 years: 2 (Jess, Will)
Older than 60 years: 0

> Finished chain.


'0-12 years: 1 (John)\n12-18 years: 1 (Sarah)\n18-30 years: 1 (Steve)\n30-60 years: 2 (Jess, Will)\nOlder than 60 years: 0'